In [1]:
import platform
import psutil
import subprocess
import socket  # Import socket module
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Create a new Word document
doc = Document()

# Helper function to add title with minimal spacing
def add_title(title):
    title_obj = doc.add_heading(level=1)
    run = title_obj.add_run(title)
    run.font.size = Pt(14)
    title_obj.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Helper function to add tables concisely
def add_table(data_dict):
    table = doc.add_table(rows=1, cols=2)
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Attribute'
    hdr_cells[1].text = 'Value'
    for key, value in data_dict.items():
        row_cells = table.add_row().cells
        row_cells[0].text = str(key)
        row_cells[1].text = str(value if value is not None else 'Unknown')

# Add basic system info
add_title('System Information')
sys_info = {
    'System': platform.system(),
    'Node Name': platform.node(),
    'Release': platform.release(),
    'Version': platform.version(),
    'Machine': platform.machine(),
    'Processor': platform.processor(),
    'Architecture': platform.architecture()
}
add_table(sys_info)

# CPU Information
add_title('CPU Information')
cpu_info = {
    'Physical Cores': psutil.cpu_count(logical=False),
    'Total Cores': psutil.cpu_count(logical=True),
    'Max Frequency (MHz)': psutil.cpu_freq().max,
    'Min Frequency (MHz)': psutil.cpu_freq().min,
    'Current Frequency (MHz)': psutil.cpu_freq().current,
    'Total CPU Usage': f'{psutil.cpu_percent()}%'
}
add_table(cpu_info)

# Memory Information (RAM)
add_title('Memory Information (RAM)')
memory_info = psutil.virtual_memory()
ram_info = {
    'Total Memory': f'{memory_info.total / (1024**3):.2f} GB',
    'Available Memory': f'{memory_info.available / (1024**3):.2f} GB',
    'Used Memory': f'{memory_info.used / (1024**3):.2f} GB',
    'Memory Usage': f'{memory_info.percent} %'
}
add_table(ram_info)

# Disk Information (HDD/SSD)
add_title('Disk Information (HDD/SSD)')
disk_info = {}
for disk in psutil.disk_partitions():
    usage = psutil.disk_usage(disk.mountpoint)
    disk_info[disk.device] = {
        'Mountpoint': disk.mountpoint,
        'File System Type': disk.fstype,
        'Total Size': f'{usage.total / (1024**3):.2f} GB',
        'Used': f'{usage.used / (1024**3):.2f} GB',
        'Free': f'{usage.free / (1024**3):.2f} GB',
        'Usage': f'{usage.percent}%'
    }

for device, info in disk_info.items():
    doc.add_heading(device, level=2)
    add_table(info)

# Get GPU information
def get_gpu_info():
    try:
        if platform.system() == "Windows":
            # Use wmic command to get GPU information on Windows
            gpu_info = subprocess.check_output("wmic path win32_videocontroller get caption, adapterram, driverversion", shell=True)
            gpu_info = gpu_info.decode().strip().split("\n")[1:]
            return [line.strip().split() for line in gpu_info]
        elif platform.system() == "Linux":
            # Use lshw or glxinfo to get GPU information on Linux
            gpu_info = subprocess.check_output("lshw -C display", shell=True)
            return gpu_info.decode().split("\n")
        else:
            return "GPU information not available for this OS."
    except Exception as e:
        return str(e)

# Add GPU info
add_title('GPU Information')
gpu_data = get_gpu_info()
if isinstance(gpu_data, str):
    doc.add_paragraph(gpu_data)
else:
    for gpu in gpu_data:
        doc.add_paragraph(' '.join(gpu))

# Get manufacturer details (using wmic for Windows)
def get_hardware_manufacturers():
    manufacturers = {}
    try:
        if platform.system() == "Windows":
            # Get manufacturer info for RAM, SSD/HDD, and CPU
            manufacturers['CPU'] = subprocess.check_output("wmic cpu get manufacturer", shell=True).decode().split("\n")[1].strip()
            manufacturers['RAM'] = subprocess.check_output("wmic memorychip get manufacturer", shell=True).decode().split("\n")[1].strip()
            manufacturers['Disk Drives'] = subprocess.check_output("wmic diskdrive get manufacturer", shell=True).decode().split("\n")[1].strip()
        elif platform.system() == "Linux":
            # Get manufacturer info using lshw or dmidecode
            manufacturers['CPU'] = subprocess.check_output("lshw -C CPU", shell=True).decode().split("\n")
            manufacturers['RAM'] = subprocess.check_output("lshw -C memory", shell=True).decode().split("\n")
            manufacturers['Disk Drives'] = subprocess.check_output("lshw -C disk", shell=True).decode().split("\n")
        else:
            manufacturers = "Manufacturer info not available for this OS."
    except Exception as e:
        manufacturers = str(e)
    return manufacturers

# Add manufacturer information
add_title('Hardware Manufacturer Information')
hardware_manufacturers = get_hardware_manufacturers()
if isinstance(hardware_manufacturers, str):
    doc.add_paragraph(hardware_manufacturers)
else:
    add_table(hardware_manufacturers)

# Network Information
add_title('Network Information')
net_info = psutil.net_if_addrs()
net_details = {}
for interface_name, addresses in net_info.items():
    for address in addresses:
        if address.family == socket.AF_INET:  # Use socket.AF_INET for IPv4
            net_details[interface_name] = {
                'IP Address': address.address,
                'Netmask': address.netmask,
                'Broadcast IP': address.broadcast
            }
        elif address.family == psutil.AF_LINK:  # Use psutil.AF_LINK for MAC addresses
            if interface_name not in net_details:
                net_details[interface_name] = {}
            net_details[interface_name]['MAC Address'] = address.address

for interface, info in net_details.items():
    doc.add_heading(interface, level=2)
    add_table(info)

# Save the document with reduced spaces
doc.save('system_info_report_reduced_spaces.docx')

print("Report generated: 'system_info_report_reduced_spaces.docx'")


Report generated: 'system_info_report_reduced_spaces.docx'
